In [ ]:
#def compare(reference, recognized):
    #matcher = difflib.SequenceMatcher(None, reference.lower().split(), recognized.lower().split())
    #matched = sum(block.size for block in matcher.get_matching_blocks())
    #total = max(len(reference.split()), len(recognized.split()))
    #return matched / total if total > 0 else 0.0


INFO:aiogram.dispatcher:Start polling
INFO:aiogram.dispatcher:Run polling for bot @sombra_spanish_bot id=7481970874 - 'SombraBot'
INFO:aiogram.event:Update id=135040789 is handled. Duration 408 ms by bot id=7481970874
INFO:aiogram.event:Update id=135040790 is handled. Duration 1762 ms by bot id=7481970874
INFO:aiogram.event:Update id=135040791 is handled. Duration 90 ms by bot id=7481970874
INFO:aiogram.event:Update id=135040792 is handled. Duration 99 ms by bot id=7481970874
INFO:aiogram.event:Update id=135040793 is handled. Duration 140 ms by bot id=7481970874
INFO:aiogram.event:Update id=135040794 is handled. Duration 2111 ms by bot id=7481970874
INFO:aiogram.dispatcher:Polling stopped for bot @sombra_spanish_bot id=7481970874 - 'SombraBot'
INFO:aiogram.dispatcher:Polling stopped


In [33]:
import os
import re
import asyncio
import nest_asyncio
from aiogram import Bot, Dispatcher, types, F
from aiogram.enums import ParseMode
from aiogram.client.default import DefaultBotProperties
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.filters import Command
from aiogram.types import FSInputFile, InlineKeyboardMarkup, InlineKeyboardButton, CallbackQuery, ReplyKeyboardMarkup, KeyboardButton
from openpyxl import load_workbook
from moviepy.editor import VideoFileClip
import whisper
import difflib
import Levenshtein
import requests


start_keyboard = ReplyKeyboardMarkup(
    keyboard=[
        [KeyboardButton(text="Обновить и перезапустить")],
        [KeyboardButton(text="Помощь")],
        [KeyboardButton(text="Выбрать фрагмент")],
        [KeyboardButton(text="Завершить на сегодня")]
        ],
    resize_keyboard=True,
    one_time_keyboard=False
)


def download_video():
    file_id = "1qhkfTq6KujEBERAVBMjRGLnb-cX4FDEO"
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    path = "Extra1.mp4"

    if not os.path.exists(path):
        print("📥 Скачиваем видео...")
        session = requests.Session()
        response = session.get(url, stream=True)
        token = get_confirm_token(response)

        if token:
            url = f"https://drive.google.com/uc?export=download&confirm={token}&id={file_id}"
            response = session.get(url, stream=True)

        save_response_content(response, path)
        print("✅ Видео скачано.")

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

download_video()

nest_asyncio.apply()

TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "7481970874:AAGtUTbF11D46zz6AtCndzZBN_rJR2TnPXg")

VIDEO_PATH = "Extra1.mp4"
FRAGMENTS_DIR = "fragments"
USER_AUDIO_DIR = "user_audio"
EXCEL_PATH = "blocks.xlsx"

total = 23

os.makedirs(FRAGMENTS_DIR, exist_ok=True)
os.makedirs(USER_AUDIO_DIR, exist_ok=True)

keywords = ['horno', 'español', 'quieres', 'llamo', 'perro', 'museo', 'americano', 'coches', 'chicas', 'vivo', 'correo',
            'dormitorio', 'ducha', 'estupendo', 'estás', 'quedar', 'tienes', 'cama', 'carta', 'chica', 'chico',
            'compras', 'dije', 'duerme', 'factura', 'gracias', 'hablar', 'queda', 'rápido', 'tomar', 'abajo', 'acabó',
            'amigos', 'años', 'baño', 'bici', 'bicicleta', 'celebrarlo', 'corresponsal', 'dices', 'digas', 'dormir',
            'echo', 'favor', 'febrero', 'fuertes', 'gusta', 'hablo', 'hombres', 'importa']

bigrams = ['el horno', 'en el', 'el perro', 'me llamo', 'un museo', 'está en', 'de américa', 'no no', 'en un',
           'perro está', 'esto es', 'muy bien', 'es la', 'el correo', 'la factura', 'bien el', 'tomar algo', 'es un',
           'quiere decir', 'con coches', 'es el', 'se vaya', 'un poco', 'perro de', 'duerme en', 'de compras',
           'compras para', 'en españa', 'le gusta', 'están las', 'una carta', 'te lo', 'se acabó', 'y no', 'no me',
           'lo siento', 'luis luis', 'que sí', 'correo luis', 'a ver', 'factura del', 'de la', 'oh es', 'es esto',
           'para ti', 'te quiere', 'américa américa', 'hace siete', 'siete años', 'español bueno']

bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher(storage=MemoryStorage())

model = whisper.load_model("base")
user_state = {}

# --- Утилиты ---

def load_clips_from_excel(path=EXCEL_PATH):
    wb = load_workbook(path)
    ws = wb.active
    return [{"start": row[0], "end": row[1], "text": row[2]} for row in ws.iter_rows(min_row=2, values_only=True)]

clips = load_clips_from_excel()

def extract_clip(video_path: str, start: float, end: float, output_path: str):
    clip = VideoFileClip(video_path).subclip(start, end)
    clip.write_videofile(output_path, codec='libx264', audio_codec='aac', logger=None)

def generate_gap_text(sentence, keywords, bigrams):
    masked = sentence
    for bg in bigrams:
        masked = re.sub(rf"\b{re.escape(bg)}\b", "_____ _____", masked, flags=re.IGNORECASE)
    for kw in keywords:
        masked = re.sub(rf"\b{re.escape(kw)}\b", "_____", masked, flags=re.IGNORECASE)
    return masked

def transcribe(audio_path):
    result = model.transcribe(audio_path, language="es")
    return result["text"]

def normalize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # удаление пунктуации
    text = re.sub(r"\s+", " ", text).strip()  # удаление лишних пробелов
    return text

def compare(reference, recognized):
    ref = normalize(reference)
    rec = normalize(recognized)
    distance = Levenshtein.distance(ref, rec)
    max_len = max(len(ref), len(rec))
    return 1 - distance / max_len if max_len > 0 else 0.0

# --- Хелпер для сброса состояния фрагмента ---

def reset_user_fragment_state(chat_id):
    if chat_id in user_state:
        user_state[chat_id]["text_attempts"] = 0
        user_state[chat_id]["attempted_text"] = False
        user_state[chat_id]["audio_sent"] = False

# --- Кнопки ---

def continue_keyboard():
    return InlineKeyboardMarkup(inline_keyboard=[
        [InlineKeyboardButton(text="✅ Продолжим?", callback_data="force_next")]
    ])

def navigation_keyboard():
    return InlineKeyboardMarkup(inline_keyboard=[
        [
            InlineKeyboardButton(text="🔙 Назад", callback_data="back"),
            InlineKeyboardButton(text="🔁 Повторить", callback_data="repeat"),
            InlineKeyboardButton(text="⏭ Пропустить", callback_data="skip")
        ]
    ])
    
def fragment_selection_keyboard():
    buttons = []
    row = []
    for i in range(1, total + 1):
        row.append(InlineKeyboardButton(text=f"Фрагмент {i}", callback_data=f"goto_{i-1}"))
        if len(row) == 5:  # по 5 кнопок в ряд
            buttons.append(row)
            row = []
    if row:
        buttons.append(row)

    return InlineKeyboardMarkup(inline_keyboard=buttons)

# --- Хендлеры ---

# --- Хендлер начала /start ---
@dp.message(F.text == "Обновить и перезапустить")
async def handle_start_text(message: types.Message):
    await cmd_start(message)

@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    user_state[message.chat.id] = {
        "index": 0,
        "attempted_text": False,
        "text_attempts": 0,
        "audio_sent": False
    }
    await message.answer("/start", reply_markup=start_keyboard)
    await send_clip(message.chat.id)

@dp.message(F.text == "Помощь")
async def handle_help_text(message: types.Message):
    await cmd_help(message)
    
@dp.message(Command("help"))
async def cmd_help(message: types.Message):
    help_text = (
        "ℹ️ <b>Помощь по боту</b>:\n\n"
        "/start — начать работу с ботом\n"
        "/help — показать это сообщение\n"
        "/select — выбрать и перейти к фрагменту сериала\n\n"
    )
    await message.answer(help_text)

@dp.message(F.text == "Выбрать фрагмент")
async def handle_goto_fragment_text(message: types.Message):
    await cmd_select(message)

@dp.message(Command("select"))
async def cmd_select(message: types.Message):
    await message.answer("Выбери номер фрагмента:", reply_markup=fragment_selection_keyboard())
    
@dp.message(F.text == "Завершить на сегодня")
async def handle_finish_today(message: types.Message):
    await message.answer("Спасибо за твою работу, ¡hasta luego!")

# --- Отправка фрагмента ---
async def send_clip(chat_id: int):
    state = user_state.get(chat_id)

    if not state or "index" not in state:
        await bot.send_message(chat_id, "⚠️ Пожалуйста, начни с команды /start.")
        return

    i = state["index"]

    if i >= len(clips):
        await bot.send_message(chat_id, "🎉 Все фрагменты завершены!")
        return

    # Сбрасываем состояние для нового фрагмента
    reset_user_fragment_state(chat_id)

    clip = clips[i]
    clip_path = os.path.join(FRAGMENTS_DIR, f"clip_{i}.mp4")

    if not os.path.exists(clip_path):
        try:
            extract_clip(VIDEO_PATH, clip['start'], clip['end'], clip_path)
        except Exception as e:
            await bot.send_message(chat_id, f"❌ Ошибка при обработке фрагмента: {e}")
            return

    video = FSInputFile(clip_path)

    await bot.send_video(chat_id=chat_id, video=video)
    await bot.send_message(
        chat_id=chat_id,
        text=(
            f"🎬 <b>Фрагмент {i + 1} из {total}</b>\n\n"
            "📝 Вставь пропущенные слова и запиши этот фрагмент целиком:\n\n"
            f"{generate_gap_text(clip['text'], keywords, bigrams)}"
        ),
        reply_markup=navigation_keyboard()
    )

# --- Обработка навигационных кнопок ---
@dp.callback_query(lambda c: c.data in ["repeat", "skip", "back"])
async def handle_navigation_buttons(callback: CallbackQuery):
    chat_id = callback.message.chat.id
    data = callback.data

    if chat_id not in user_state:
        await callback.answer("Пожалуйста, начни с команды /start")
        return

    state = user_state[chat_id]

    if data == "repeat":
        await callback.message.answer("🔁 Повторим текущий фрагмент:")
        await send_clip(chat_id)
    elif data == "skip":
        state["index"] += 1
        reset_user_fragment_state(chat_id)
        await callback.message.answer("⏭ Пропущено. Следующий фрагмент:")
        await send_clip(chat_id)
    elif data == "back":
        state["index"] = max(0, state["index"] - 1)
        reset_user_fragment_state(chat_id)
        await callback.message.answer("🔙 Вернулись к предыдущему фрагменту:")
        await send_clip(chat_id)

    await callback.answer()
    
@dp.callback_query(lambda c: c.data and c.data.startswith("goto_"))
async def handle_goto_fragment(callback: CallbackQuery):
    chat_id = callback.message.chat.id
    if chat_id not in user_state:
        await callback.answer("Пожалуйста, начни с команды /start")
        return
    
    fragment_index = int(callback.data.split("_")[1])
    if fragment_index < 0 or fragment_index >= len(clips):
        await callback.answer("Некорректный номер фрагмента")
        return
    
    user_state[chat_id]["index"] = fragment_index
    reset_user_fragment_state(chat_id)
    
    await callback.message.answer(f"Переключаемся на фрагмент {fragment_index + 1}")
    await send_clip(chat_id)
    await callback.answer()

# --- Обработка кнопки "Продолжим?" ---
@dp.callback_query(lambda c: c.data == "force_next")
async def handle_force_next(callback: CallbackQuery):
    chat_id = callback.message.chat.id

    if chat_id not in user_state:
        await callback.answer("Пожалуйста, начни с команды /start")
        return

    state = user_state[chat_id]

    if not state.get("audio_sent", False):
        # Пользователь ещё не прислал аудио — предлагаем прислать
        await callback.message.answer(
            "🎤 Теперь пришли аудиозапись с этим фрагментом."
        )
        await callback.answer()
    else:
        # Аудио уже есть — переходим к следующему фрагменту
        state["index"] += 1
        reset_user_fragment_state(chat_id)
        await callback.message.answer("⏭ Продолжаем и переходим к следующему фрагменту.")
        await send_clip(chat_id)
        await callback.answer()
        
@dp.callback_query(lambda c: c.data == "show_answer")
async def show_answer_handler(callback: CallbackQuery):
    chat_id = callback.message.chat.id
    if chat_id not in user_state:
        await callback.answer("Пожалуйста, начни с команды /start")
        return

    state = user_state[chat_id]
    i = state["index"]
    correct_answer = clips[i]["text"]

    await callback.message.answer(f"✅ Правильный ответ:\n\n<b>{correct_answer}</b>", reply_markup=continue_keyboard())
    await callback.answer()
        
@dp.message(F.text)
async def handle_text_response(message: types.Message):
    chat_id = message.chat.id
    if chat_id not in user_state:
        return
    
    state = user_state[chat_id]
    i = state["index"]
    correct = clips[i]["text"].lower().strip()
    answer = message.text.lower().strip()

    state["attempted_text"] = True
    state["text_attempts"] = state.get("text_attempts", 0) + 1  # увеличиваем счётчик

    if answer == correct:
        state["text_attempts"] = 0  # сбрасываем при успехе
        await message.answer("✅ Верно! Теперь запиши аудио с этим предложением.")
    else:
        if state["text_attempts"] >= 2:
            keyboard = InlineKeyboardMarkup(inline_keyboard=[
                [InlineKeyboardButton(text="👀 Показать правильный ответ", callback_data="show_answer")]
            ])
            await message.answer(
                "❌ Ошибка. Ты можешь попробовать ещё раз или нажать кнопку ниже, чтобы увидеть правильный ответ.",
                reply_markup=keyboard
            )
        else:
            await message.answer(
                "⚠️ Есть ошибки в тексте. Попробуй ещё раз или отправь голосовое сообщение с произношением.")

# --- Обработка аудио ---
@dp.message(F.voice | F.audio)
async def handle_audio(message: types.Message):
    chat_id = message.chat.id

    if chat_id not in user_state:
        await message.answer("Пожалуйста, начни с команды /start")
        return

    i = user_state[chat_id]["index"]

    voice = message.voice or message.audio
    file = await bot.get_file(voice.file_id)

    audio_path = os.path.join(USER_AUDIO_DIR, f"audio_{chat_id}_{i}.ogg")
    await bot.download_file(file.file_path, audio_path)

    recognized = transcribe(audio_path)
    reference = clips[i]["text"]
    score = compare(reference, recognized)

    await message.answer(
        f"🔊 Ты сказал: <i>{recognized}</i>\n📊 Произношение: <b>{score*100:.1f}%</b>"
    )

    state = user_state[chat_id]
    state["audio_sent"] = True  # Отмечаем, что аудио отправлено

    if score <= 0.7:
        keyboard = InlineKeyboardMarkup(inline_keyboard=[
            [InlineKeyboardButton(text="🔁 Попробовать ещё раз", callback_data="repeat_audio")],
            [InlineKeyboardButton(text="⏭ Продолжить несмотря на ошибки", callback_data="force_next")]
        ])
        await message.answer("😕 Произношение неточное. Хочешь попробовать ещё раз или перейти дальше?", reply_markup=keyboard)
    else:
        # Произношение хорошее — предлагаем перейти дальше
        await message.answer("👍 Хорошо! Теперь нажми кнопку ниже, чтобы перейти к следующему фрагменту.", reply_markup=continue_keyboard())

# --- Повтор аудио ---
@dp.callback_query(lambda c: c.data == "repeat_audio")
async def repeat_audio(callback: CallbackQuery):
    chat_id = callback.message.chat.id
    await callback.message.answer("🎤 Пришли, пожалуйста, аудио с повторной записью фрагмента.")
    await callback.answer()

async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())


INFO:aiogram.dispatcher:Start polling
INFO:aiogram.dispatcher:Run polling for bot @sombra_spanish_bot id=7481970874 - 'SombraBot'
INFO:aiogram.event:Update id=135040910 is handled. Duration 2875 ms by bot id=7481970874
INFO:aiogram.event:Update id=135040911 is handled. Duration 82 ms by bot id=7481970874
INFO:aiogram.dispatcher:Polling stopped for bot @sombra_spanish_bot id=7481970874 - 'SombraBot'
INFO:aiogram.dispatcher:Polling stopped
